In [1]:
from manim import *

In [ ]:
import math
import numpy as np
from collections import defaultdict
import random

In [ ]:
# Celda 1  En Google Colab
# !apt update
# !apt install -y ffmpeg libcairo2-dev libpango1.0-dev
# Celda 2
# !pip install manim
# Celda 3
# !pip install numpy==1.26.4 --force-reinstall
# Celda 4
# !apt-get update
# !apt-get install texlive texlive-latex-extra texlive-fonts-recommended dvipng cm-super -y

# Celda 5 o subsiguientes
#from PIL import Image (En Colab)
#config.background_color = "#24043d"
# from google.colab import files
# uploaded = files.upload()


In [2]:
import sys
from pathlib import Path

# Subir un solo nivel desde la carpeta actual (escenas/horizontales)
root_path = Path().resolve().parents[1]    # parents[0] sería un salto hacia arriba, [1] son dos

# Agregarlo al sys.path si no está ya
if str(root_path) not in sys.path:
    sys.path.append(str(root_path))

In [3]:
import src.herramientas as h

In [4]:
# from layouts.basicos import en_linea_horizontal
# import src.funciones as f

In [5]:
config.media_width = "75%"
config.verbosity = "WARNING"
config.background_color = "#24043d"

In [9]:
%%manim -ql MiClase
class MiClase(Scene): 
#class MiClase(MovingCameraScene):
    
    def construct(self):
        # Obtener el tamaño de la pantalla
        ancho = self.camera.frame_width
        alto = self.camera.frame_height
    
        #Añadir logo
        logo = h.Logo()
        self.add(logo)

        grilla = h.GrillaRegla()
        self.add(grilla)
        self.wait(5)

Manim Community v0.19.0

In [ ]:
%%manim -ql -v WARNING DistribucionElectronicaECuatro

subniveles = [
    ("1s", 2), ("2s", 2), ("2p", 6),
    ("3s", 2), ("3p", 6),
    ("4s", 2), ("3d", 10), ("4p", 6),
    ("5s", 2), ("4d", 10), ("5p", 6),
    ("6s", 2), ("4f", 14),("5d", 10),
    ("6p", 6),("7s", 2),("5f", 14),
    ("6d", 10),("7p", 6)
]


def crear_orbitas(n: int, centro=ORIGIN, radio_inicial: float = 0.55, paso: float = 0.3, color=BLUE) -> list:
    return [
        Circle(radius=radio_inicial + i * paso, color=color, stroke_width=2, fill_opacity=0).move_to(centro)
        for i in range(n)
    ]


def ubicar_electrones(n: int, radio: float, centro=ORIGIN, color=WHITE) -> VGroup:
    return VGroup(*[
        Circle(radius=0.08, color=color, fill_opacity=0.6, fill_color=color)
        .move_to(centro + radio * np.array([np.cos(theta + PI/2), np.sin(theta + PI/2), 0]))
        for theta in np.linspace(0, TAU, n, endpoint=False)
    ])


def distribucion_electronica(n_electrones: int) -> dict:
    distribucion = {}
    for subnivel, capacidad in subniveles:
        if n_electrones == 0:
            break
        cantidad = min(capacidad, n_electrones)
        distribucion[subnivel] = cantidad
        n_electrones -= cantidad
    return distribucion

def contar_e_por_nivel(distribucion: dict) -> dict:
    niveles = defaultdict(int)
    for subnivel, cantidad in distribucion.items():
        nivel = int(subnivel[0])
        niveles[nivel] += cantidad
    return dict(niveles)

def en_grilla(grupo: VGroup, columnas: int, espacio_x: float = 1.0, espacio_y: float = 1.0,
              alineacion_horizontal: str = "centro", alineacion_vertical: str = "centro",
              max_izquierda: float = -6, max_derecha: float = 6,
              max_abajo: float = -3.5, max_arriba: float = 3.5,
              offset_x: float = 0, offset_y: float = 0) -> VGroup:
    """
    Distribuye un grupo de objetos en una grilla, organizados en filas y columnas,
    dentro de límites definidos, con espacio entre objetos y opciones de alineación.

    Parámetros:
    - grupo: VGroup con los objetos a distribuir
    - columnas: cantidad de columnas deseadas
    - espacio_x: espacio horizontal entre objetos
    - espacio_y: espacio vertical entre objetos
    - alineacion_horizontal: 'izquierda', 'centro' o 'derecha'
    - alineacion_vertical: 'arriba', 'centro' o 'abajo'
    - max_izquierda / max_derecha: límites horizontales
    - max_abajo / max_arriba: límites verticales
    - offset_x / offset_y: corrimientos adicionales

    Retorna:
    - El mismo grupo, con objetos reposicionados
    """
    n = len(grupo)
    filas = (n + columnas - 1) // columnas

    # Posicionar los objetos en forma de grilla
    for idx, obj in enumerate(grupo):
        fila = idx // columnas
        col = idx % columnas
        obj.move_to([col * espacio_x, -fila * espacio_y, 0])

    # Calculamos el ancho y alto de la grilla
    ancho_total = (min(columnas, n) - 1) * espacio_x
    alto_total = (filas - 1) * espacio_y

    # Alineación horizontal
    if alineacion_horizontal == "centro":
        grupo.shift(RIGHT * ((max_derecha + max_izquierda - ancho_total) / 2 + offset_x))
    elif alineacion_horizontal == "izquierda":
        grupo.shift(LEFT * abs(max_izquierda) + RIGHT * offset_x)
    elif alineacion_horizontal == "derecha":
        grupo.shift(RIGHT * abs(max_derecha - ancho_total) + RIGHT * offset_x)

    # Alineación vertical
    if alineacion_vertical == "centro":
        grupo.shift(UP * ((max_arriba + max_abajo - alto_total) / 2 + offset_y))
    elif alineacion_vertical == "arriba":
        grupo.shift(UP * abs(max_arriba) + UP * offset_y)
    elif alineacion_vertical == "abajo":
        grupo.shift(DOWN * abs(max_abajo - alto_total) + UP * offset_y)

    return grupo

def crear_cuadro_elemento(simbolo_str, nombre_str, z, masa_str, tipo_str, carga_str, color_fondo="#88D5EB", color_interior="#81B2EB"):
    exterior = RoundedRectangle(width=5, height=5, corner_radius=0.5, color=color_interior, fill_opacity=1)
    interior = RoundedRectangle(width=4.8, height=4.8, corner_radius=0.48, color=color_fondo, fill_opacity=1).move_to(exterior)
    simbolo = Text(simbolo_str, font="Monospace", font_size=106, color=BLACK).move_to(interior.get_center() + 0.25 * UP)
    nombre = Text(nombre_str, font="Monospace", font_size=30, color=BLACK).move_to(simbolo.get_bottom() + 0.45 * DOWN)
    masa = Text("Masa\nAtómica", font="Monospace", font_size=24, color=BLACK).move_to(interior.get_corner(UL) + DOWN * 0.35 + RIGHT * 0.95)
    magnitud_masa = Text(masa_str, font="Monospace", font_size=28, color=BLACK).next_to(masa.get_center() + 0.6 * DOWN + 1 * LEFT)
    zeta = Text("Número\nAtómico", font="Monospace", font_size=24, color=BLACK).move_to(interior.get_corner(UR) + DOWN * 0.35 + LEFT * 0.75)
    numero_atomico = Text(str(z), font="Monospace", font_size=28, color=BLACK).next_to(zeta.get_center() + 0.6 * DOWN + 0.3 * LEFT)
    tipo = Text(tipo_str, font="Monospace", font_size=24, color=BLACK).move_to(interior.get_corner(DL) + UP * 0.35 + RIGHT * 1.15)
    electronegatividad = Text(carga_str, font="Monospace", font_size=24, color=BLACK).move_to(interior.get_corner(DR) + UP * 0.35 + LEFT * 1)

    return Group(exterior, interior, simbolo, nombre, masa, magnitud_masa, zeta, numero_atomico, tipo, electronegatividad)


def crear_etiqueta(x, nivel, width=3.25, height=0.6):
        if x == 1:
            texto = Text(f"{x} electrón en nivel {nivel}", font="Liberation Sans", font_size=42, weight=BOLD , color=BLACK).scale(0.45)
        else:
            texto = Text(f"{x} electrones en nivel {nivel}", font="Liberation Sans", font_size=42, weight=BOLD , color=BLACK).scale(0.45)
        fondo = RoundedRectangle(corner_radius=0.2, width=width, height=height, color=BLACK, fill_color=WHITE, fill_opacity=1)
        etiqueta = VGroup(fondo, texto)
        texto.move_to(fondo.get_center())
        return etiqueta

def crear_circulo_sombreado(
    radio=1,
    color=RED_B,
    opacidad_relleno=0.8,
    sheen=0.5,
    stroke_color=WHITE,
    stroke_width=1,
    stroke_opacidad=0  # valor por defecto como pediste
):
    circulo = Circle(radius=radio)
    circulo.set_fill(color=color, opacity=opacidad_relleno)
    circulo.set_sheen(sheen)
    circulo.set_stroke(color=stroke_color, width=stroke_width, opacity=stroke_opacidad)
    return circulo

def crear_icono_alerta():
    # Triángulo equilátero como base
    triangulo = Triangle()
    #triangulo.set_fill(RED, opacity=0.5)
    triangulo.set_stroke(RED, width=4)
    triangulo.scale(1.25)

    triangulo2 = triangulo.copy().scale(0.9).shift(0.04*DOWN)

    # Signo de exclamación
    signo = Text("!", font_size=90, color=YELLOW_B)
    signo.move_to(triangulo.get_center() + 0.15 * DOWN)

    # Agrupar todo
    simbolo_alerta = VGroup(triangulo, triangulo2 ,signo)

    return simbolo_alerta


def crear_liston_hexagonal():
    # Puntos del hexágono regular (antes de rotación y escala)
    puntos = [
        UP * 2,
        UP * 1 + RIGHT * math.sqrt(3),
        DOWN * 1 + RIGHT * math.sqrt(3),
        DOWN * 2,
        DOWN * 1 + LEFT * math.sqrt(3),
        UP * 1 + LEFT * math.sqrt(3),
    ]

    # Función auxiliar para crear hexágonos ya transformados
    def crear_hexagono(color, desplazamiento):
        return Polygon(*puntos,
                       fill_color=color,
                       fill_opacity=1,
                       stroke_opacity=0
               ).rotate(30*DEGREES).scale(0.4).shift(desplazamiento)

    offset_x = 0.3

    # Crear hexágonos
    hexagono_relleno = crear_hexagono(BLUE, (4 + offset_x) * LEFT)
    # hexágono en el extremo derecho del listón, sobre este se cierra el rectángulo que forma el listón.
    hexagono_relleno2 = crear_hexagono(WHITE, 5*RIGHT)
    # hexágono que ayuda a crear una de las flechas
    hexagono_base = crear_hexagono(WHITE, 4*LEFT)

    # Flechas (recortes entre hexágonos)
    flecha = Difference(hexagono_base, hexagono_relleno)
    flecha.set_fill(BLUE, opacity=1)
    flecha.set_stroke(opacity=0)
    flecha.shift(offset_x * RIGHT)

    flecha2 = flecha.copy().shift(0.30 * RIGHT).set_fill(config.background_color)

    # Rectángulo blanco intermedio
    rect_intermedio = Polygon(
        flecha2.get_top(),
        hexagono_relleno2.get_top(),
        hexagono_relleno2.get_bottom(),
        flecha2.get_bottom(),
        fill_color=WHITE,
        fill_opacity=1,
        stroke_opacity=0
    )

    # Z-INDEX explícito
    hexagono_relleno.set_z_index(-2)
    hexagono_relleno2.set_z_index(-2)
    flecha.set_z_index(-1)
    flecha2.set_z_index(-1)
    rect_intermedio.set_z_index(-3)

    # Agrupación final
    panel = VGroup(hexagono_relleno, rect_intermedio, hexagono_relleno2, flecha, flecha2)
    panel.set_z_index(-4)
    return panel



# 1111111111111111111111111111111111111111111111111111111111111111111111111111111111

class DistribucionElectronicaEUno(Scene):

    def construct(self):

        #Añadir logo
        logo = h.Logo()
        self.add(logo)

        ###########################
        #                         #
        #      Tabla niveles      #
        #      y subniveles       #
        #                         #
        ###########################

        ancho_rectangulo = 0.9
        alto_rectangulo = 0.625

      # Crear 8 para la primera columna
        para_niveles = VGroup(*[
            Rectangle(width=ancho_rectangulo, height=alto_rectangulo, color=WHITE)
            for _ in range(8)
        ])

        col_niveles = en_grilla(para_niveles,
            columnas=1,
            espacio_x=ancho_rectangulo,   # Si el epacio que le doy es = al ancho del rectángulo, entonces quedan pegados uno al lado del otro
            espacio_y=alto_rectangulo,
            alineacion_horizontal="izquierda",
            alineacion_vertical="arriba",
            max_izquierda=-6.15,
            max_derecha=6,
            max_abajo=-3.5,
            max_arriba=3.3)

      # Crear 4 columnas para los subniveles
        para_subniveles = VGroup(*[
            Rectangle(width=ancho_rectangulo, height=alto_rectangulo, color=WHITE)
            for _ in range(32)
        ])

        cols_subniveles = en_grilla(para_subniveles,
            columnas=4,
            espacio_x=ancho_rectangulo,
            espacio_y=alto_rectangulo,
            alineacion_horizontal="izquierda",
            alineacion_vertical="arriba",
            max_izquierda=-6.15,
            max_derecha=6,
            max_abajo=-3.5,
            max_arriba=3.3).shift((ancho_rectangulo) * RIGHT)

        tabla = Group(col_niveles, cols_subniveles)
        tabla.shift(0.25*LEFT)

        # Texto a casillas
        tags_niveles= ["","1","2","3","4","5","6","7"]
        tags_niveles_alineados= Group()

        for i in range(len(tags_niveles)):
          tag_nivel = Text(tags_niveles[i], font="Monospace", font_size=20 ).move_to(col_niveles[i].get_center())
          tags_niveles_alineados.add(tag_nivel)
        tabla.add(tags_niveles_alineados)             # Agrego los tags a la tabla


        tags_subniveles = [
                              Tex(r"$s$",font_size=38).set(z_index=600), Tex(r"$p$",font_size=38).set(z_index=601), Tex(r"$d$",font_size=38).set(z_index=602), Tex(r"$f$",font_size=38).set(z_index=603),
                              Tex(r"$1s^2$",font_size=38).set(z_index=605), Tex(r"",font_size=38).set(z_index=606), Tex(r"",font_size=38).set(z_index=607), Tex(r"",font_size=38).set(z_index=608),  # vacíos pero válidos como raw strings
                              Tex(r"$2s^2$",font_size=38).set(z_index=609), Tex(r"$2p^6$",font_size=38).set(z_index=610), Tex(r"",font_size=38).set(z_index=611), Tex(r"",font_size=38).set(z_index=612),
                              Tex(r"$3s^2$",font_size=38).set(z_index=613), Tex(r"$3p^6$",font_size=38).set(z_index=614), Tex(r"$3d^{10}$",font_size=38).set(z_index=615), Tex(r"",font_size=38).set(z_index=616),
                              Tex(r"$4s^2$",font_size=38).set(z_index=617), Tex(r"$4p^6$",font_size=38).set(z_index=618), Tex(r"$4d^{10}$",font_size=38).set(z_index=619), Tex(r"$4f^{14}$",font_size=38).set(z_index=620),
                              Tex(r"$5s^2$",font_size=38).set(z_index=621), Tex(r"$5p^6$",font_size=38).set(z_index=622), Tex(r"$5d^{10}$",font_size=38).set(z_index=623), Tex(r"$5f^{14}$",font_size=38).set(z_index=624),
                              Tex(r"$6s^2$",font_size=38).set(z_index=625), Tex(r"$6p^6$",font_size=38).set(z_index=626), Tex(r"$6d^{10}$",font_size=38).set(z_index=627), Tex(r"",font_size=38).set(z_index=628),
                              Tex(r"$7s^2$",font_size=38).set(z_index=629), Tex(r"$7p^6$",font_size=38).set(z_index=630), Tex(r"",font_size=38).set(z_index=631), Tex(r"",font_size=38).set(z_index=632)
                          ]

        tags_subniveles_alineados = VGroup()
        for i in range(len(tags_subniveles)):
            tag = tags_subniveles[i].copy().move_to(cols_subniveles[i].get_center())
            tags_subniveles_alineados.add(tag)



        # Colorear casillas
        colores = ["#CDE0B8", "#E0C1B8", "#B8C1E0", "#5A658B", "#49612F", "#613A2F", "#285FDE", GRAY, "#2A6114", "#B1E09F"]

        # Colorear la tabla según diagonales
        for i in range(4, len(cols_subniveles)):  # empezar en 4 para saltar la primera fila
            idx_relativo = i - 4  # posición dentro de la tabla real (sin títulos)
            fila = idx_relativo // 4
            col = idx_relativo % 4
            diagonal = fila + col
            color = colores[diagonal % len(colores)]

            cols_subniveles[i].set_fill(color, opacity=0.5)

        nucleo = crear_circulo_sombreado(0.28, RED_B, 0.8, 0.5, WHITE, 1, 0).shift(1.75*RIGHT + 0.5 * DOWN)

        #########################################
        #                                       #
        #     Lista para jugar con la tabla     #
        #                                       #
        #########################################
        lista_para_jugar = []

        for i in range(8):  # hasta col_niveles[7] inclusive
            lista_para_jugar.append(col_niveles[i])
            lista_para_jugar.extend(cols_subniveles[i * 4 : i * 4 + 4])


        ###############################
        #                             #
        #        Modelo de Bohr       #
        #        simplificado         #
        #                             #
        ###############################

        subtitulo_bohr = Text("Modelo de Bohr simplificado", font="Liberation Sans", font_size=42).shift(3.25 * UP)

        contador = 1
        orbitas_muestras = VGroup(*crear_orbitas(7, nucleo.get_center(), 0.6, 0.3, color=BLUE))
        niveles_permanentes= Group()
        self.wait(3)
        self.play(Write(subtitulo_bohr))
        self.wait(4)

        alerta = crear_icono_alerta().shift(0.75*UP)



        liston_uno = crear_liston_hexagonal().shift(1.75*UP + 14*RIGHT)
        liston_dos = liston_uno.copy().shift(2.25*DOWN + 1 * RIGHT)
        liston_tres = liston_dos.copy().shift(2.25*DOWN + 1 * RIGHT)

        numero1 = Text("1", font="Liberation Sans", font_size=36, color=WHITE).move_to(liston_uno[0].get_center())
        numero2 = Text("2", font="Liberation Sans", font_size=36, color=WHITE).move_to(liston_dos[0].get_center())
        numero3 = Text("3", font="Liberation Sans", font_size=36, color=WHITE).move_to(liston_tres[0].get_center())

        self.add(numero1, numero2, numero3)


        idea_principal1 = Text("El átomo tiene un núcleo central con\nprotones y neutrones",
                              font = "Liberation Sans", font_size=30, color=BLACK)

        idea_principal2 = Text('Los electrones orbitan alrededor del\nnúcleo en distintos "niveles de energía"', font="Liberation Sans", font_size=30, color=BLACK)
        idea_principal3 = Text("Cada nivel puede contener un\nnúmero limitado de electrones", font="Liberation Sans", font_size=30, color=BLACK)

        self.play(FadeIn(alerta))
        self.play(Wiggle(alerta))
        self.wait(1)
        self.play(FadeOut(alerta))

        self.wait(1)
        # Posiciono los listones
        self.play(liston_uno.animate.shift(15.25*LEFT), numero1.animate.shift(15.25*LEFT), run_time = 0.75)
        self.wait(0.66)
        self.play(liston_dos.animate.shift(15.25*LEFT), numero2.animate.shift(15.25*LEFT), run_time = 0.75)
        self.wait(0.66)
        self.play(liston_tres.animate.shift(15.25*LEFT), numero3.animate.shift(15.25*LEFT), run_time = 0.75)
        self.wait(0.66)

        # muevo el texto de las ideas a su posición en el texto
        idea_principal1.move_to(liston_uno.get_center() + 1 * RIGHT)
        idea_principal2.move_to(liston_dos.get_center() + 1 * RIGHT)
        idea_principal3.move_to(liston_tres.get_center() + 0.25 * RIGHT)

        self.play(Write(idea_principal1))
        self.wait(2)
        self.play(Write(idea_principal2))
        self.wait(4)
        self.play(Write(idea_principal3))
        self.wait(6)

        en_panel = VGroup(liston_uno, liston_dos, liston_tres,
                          idea_principal1, idea_principal2, idea_principal3,
                          numero1, numero2, numero3)

        # Quito los listones de la pantalla
        self.play(en_panel.animate.shift(15.25*LEFT), subtitulo_bohr.animate.shift(15.25*LEFT))
        self.remove(en_panel, subtitulo_bohr)
        self.wait(2)


        for elemento in lista_para_jugar:
            self.play(FadeIn(elemento), run_time=0.35)
        self.wait(2)


        self.play(GrowFromCenter(nucleo))
        self.wait(9)

        # Crear niveles temporales para mostrar que una fila de la tabla se corresponde con un nivel de energía
        for orbita in orbitas_muestras:
            nivel_temp = Group(tags_niveles_alineados[contador], col_niveles[ contador],
                               cols_subniveles[contador * 4], cols_subniveles[contador * 4 + 1],
                               cols_subniveles[contador * 4 + 2], cols_subniveles[contador * 4 + 3])
            niveles_permanentes.add(nivel_temp)
            if contador == 1 or contador == 2:
                self.play(nivel_temp.animate.shift(1*RIGHT))
                self.wait(2)
                self.play(GrowFromCenter(orbita))
                self.wait(2)
                self.play(nivel_temp.animate.shift(1*LEFT))

            else:
                self.play(nivel_temp.animate.shift(1*RIGHT), run_time=0.75)
                self.play(GrowFromCenter(orbita), run_time = 0.75)
                self.play(nivel_temp.animate.shift(1*LEFT), run_time=0.75)

            contador += 1
        contador = 0

        self.wait(4)

        # Crear los rectángulos de resaltado
        framebox1 = SurroundingRectangle(cols_subniveles[0], buff=0.1)
        framebox2 = SurroundingRectangle(cols_subniveles[1], buff=0.1)
        framebox3 = SurroundingRectangle(cols_subniveles[2], buff=0.1)
        framebox4 = SurroundingRectangle(cols_subniveles[3], buff=0.1)

        # headers de subniveles
        for i in range(4):
            self.play(Write(tags_subniveles_alineados[i]))
            if i == 0:
                self.play(Create(framebox1))
                self.wait()
            elif i == 1:
              self.play(ReplacementTransform(framebox1, framebox2))
              self.wait()
            elif i == 2:
              self.play(ReplacementTransform(framebox2, framebox3))
              self.wait()
            elif i == 3:
              self.play(ReplacementTransform(framebox3, framebox4))
              self.wait(2)

        self.play(FadeOut(framebox4))
        self.wait(12)


        #self.play(Write(tags_subniveles_alineados[1]))

        # escribo los subniveles en pantalla
        for i in range(4, len(tags_subniveles_alineados)):
            if tags_subniveles_alineados[i] == "":
                continue
            self.play(Write(tags_subniveles_alineados[i]), run_time=0.8)
            if i == 4:
                self.wait(45)
        # Una vez estén visibles, los agrego
        tabla.add(tags_subniveles_alineados)

        # Mostrar máximo de electrones por nivel

        columna_total_electrones = VGroup(*[
            Rectangle(width=4, height=alto_rectangulo, color=BLUE)
            for _ in range(8)
        ])

        self.play(ShrinkToCenter(orbitas_muestras), nucleo.animate.set_opacity(0), run_time = 2.5)

        # Tabla para mostrar total de electrones en cada nivel

        total_por_nivel = en_grilla(columna_total_electrones,
            columnas=1,
            espacio_x=2,   # Si el epacio que le doy es = al ancho del rectángulo, entonces quedan pegados uno al lado del otro
            espacio_y=alto_rectangulo,
            alineacion_horizontal="izquierda",
            alineacion_vertical="arriba",
            max_izquierda=-6.15,
            max_derecha=6,
            max_abajo=-3.5,
            max_arriba=3.3)

        total_por_nivel.next_to(tabla).shift(0.2*LEFT)
        self.add(total_por_nivel)

        texto_electrones_por_nivel = Text("Max. electrones por nivel", font="Liberation Sans", font_size = 24).move_to(total_por_nivel[0].get_center())
        self.play(Write(texto_electrones_por_nivel))
        lista_total_electrones = ["2","8","18","32","32","18","8"]
        grupo_total_electrones = Group()
        grupo_total_electrones.add(texto_electrones_por_nivel)
        for i in range(7):
            text_cantidad = Text(lista_total_electrones[i], font="Liberation Sans", font_size=26).move_to(total_por_nivel[int(i)+1].get_center())
            grupo_total_electrones.add(text_cantidad)
            self.play(FadeIn(text_cantidad))
            self.wait(2)


        # Desaparecemos el modelo de órbitas

        self.wait(19)
        self.play(FadeOut(total_por_nivel),FadeOut(grupo_total_electrones))

# 2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222

class DistribucionElectronicaEDos(Scene):

    def construct(self):

        #Añadir logo
        logo = h.Logo()
        self.add(logo)

        ###########################
        #                         #
        #      Tabla niveles      #
        #      y subniveles       #
        #                         #
        ###########################

        ancho_rectangulo = 0.9
        alto_rectangulo = 0.625

      # Crear 8 para la primera columna
        para_niveles = VGroup(*[
            Rectangle(width=ancho_rectangulo, height=alto_rectangulo, color=WHITE)
            for _ in range(8)
        ])

        col_niveles = en_grilla(para_niveles,
            columnas=1,
            espacio_x=ancho_rectangulo,   # Si el epacio que le doy es = al ancho del rectángulo, entonces quedan pegados uno al lado del otro
            espacio_y=alto_rectangulo,
            alineacion_horizontal="izquierda",
            alineacion_vertical="arriba",
            max_izquierda=-6.15,
            max_derecha=6,
            max_abajo=-3.5,
            max_arriba=3.3)

      # Crear 4 columnas para los subniveles
        para_subniveles = VGroup(*[
            Rectangle(width=ancho_rectangulo, height=alto_rectangulo, color=WHITE)
            for _ in range(32)
        ])

        cols_subniveles = en_grilla(para_subniveles,
            columnas=4,
            espacio_x=ancho_rectangulo,
            espacio_y=alto_rectangulo,
            alineacion_horizontal="izquierda",
            alineacion_vertical="arriba",
            max_izquierda=-6.15,
            max_derecha=6,
            max_abajo=-3.5,
            max_arriba=3.3).shift((ancho_rectangulo) * RIGHT)

        tabla = Group(col_niveles, cols_subniveles)
        tabla.shift(0.25*LEFT)

        # Texto a casillas
        tags_niveles= ["","1","2","3","4","5","6","7"]
        tags_niveles_alineados= Group()

        for i in range(len(tags_niveles)):
          tag_nivel = Text(tags_niveles[i], font="Monospace", font_size=20 ).move_to(col_niveles[i].get_center())
          tags_niveles_alineados.add(tag_nivel)
        tabla.add(tags_niveles_alineados)             # Agrego los tags a la tabla

        tags_subniveles = [
                              Tex(r"$s$",font_size=38).set(z_index=600), Tex(r"$p$",font_size=38).set(z_index=601), Tex(r"$d$",font_size=38).set(z_index=602), Tex(r"$f$",font_size=38).set(z_index=603),
                              Tex(r"$1s^2$",font_size=38).set(z_index=605), Tex(r"",font_size=38).set(z_index=606), Tex(r"",font_size=38).set(z_index=607), Tex(r"",font_size=38).set(z_index=608),  # vacíos pero válidos como raw strings
                              Tex(r"$2s^2$",font_size=38).set(z_index=609), Tex(r"$2p^6$",font_size=38).set(z_index=610), Tex(r"",font_size=38).set(z_index=611), Tex(r"",font_size=38).set(z_index=612),
                              Tex(r"$3s^2$",font_size=38).set(z_index=613), Tex(r"$3p^6$",font_size=38).set(z_index=614), Tex(r"$3d^{10}$",font_size=38).set(z_index=615), Tex(r"",font_size=38).set(z_index=616),
                              Tex(r"$4s^2$",font_size=38).set(z_index=617), Tex(r"$4p^6$",font_size=38).set(z_index=618), Tex(r"$4d^{10}$",font_size=38).set(z_index=619), Tex(r"$4f^{14}$",font_size=38).set(z_index=620),
                              Tex(r"$5s^2$",font_size=38).set(z_index=621), Tex(r"$5p^6$",font_size=38).set(z_index=622), Tex(r"$5d^{10}$",font_size=38).set(z_index=623), Tex(r"$5f^{14}$",font_size=38).set(z_index=624),
                              Tex(r"$6s^2$",font_size=38).set(z_index=625), Tex(r"$6p^6$",font_size=38).set(z_index=626), Tex(r"$6d^{10}$",font_size=38).set(z_index=627), Tex(r"",font_size=38).set(z_index=628),
                              Tex(r"$7s^2$",font_size=38).set(z_index=629), Tex(r"$7p^6$",font_size=38).set(z_index=630), Tex(r"",font_size=38).set(z_index=631), Tex(r"",font_size=38).set(z_index=632)
                          ]

        tags_subniveles_alineados = VGroup()
        for i in range(len(tags_subniveles)):
            tag = tags_subniveles[i].copy().move_to(cols_subniveles[i].get_center())
            tags_subniveles_alineados.add(tag)

        tabla.add(tags_subniveles_alineados)

        # Colorear casillas

        colores = ["#CDE0B8", "#E0C1B8", "#B8C1E0", "#5A658B", "#49612F", "#613A2F", "#285FDE", GRAY, "#2A6114", "#B1E09F"]


        for i in range(4, len(cols_subniveles)):  # empezar en 4 para saltar la primera fila
            idx_relativo = i - 4  # posición dentro de la tabla real (sin títulos)
            fila = idx_relativo // 4
            col = idx_relativo % 4
            diagonal = fila + col
            color = colores[diagonal % len(colores)]

            cols_subniveles[i].set_fill(color, opacity=0.5)

        self.add(tabla)

        ###########################
        #                         #
        #      Modelo orbital     #
        #                         #
        ###########################

        nucleo = crear_circulo_sombreado(0.28, RED_B, 0.8, 0.5, WHITE, 1, 0).shift(1.15 * DOWN + 1.15*RIGHT)

        contador = 1
        orbitas_muestras = VGroup(*crear_orbitas(7, nucleo.get_center(), 0.6, 0.3, color=BLUE))
        niveles_permanentes= Group()

        self.wait(1)

        flechas = VGroup()
        flecha_original = Arrow(start=cols_subniveles[11].get_corner(UR),end=col_niveles[6].get_corner(DL),stroke_width=3,
                                   stroke_opacity=1, fill_opacity=1)  # aplica a la punta)


        for i in range(len(colores)):
            flecha_temp = flecha_original.copy()
            flechas.add(flecha_temp)
        # Agrego las flechas al grupo
        flechas[0].shift(3*alto_rectangulo * UP + ancho_rectangulo*LEFT).scale(0.575)
        flechas[1].shift(2.5* alto_rectangulo* UP + 0.5 * ancho_rectangulo * LEFT ).scale(0.8)
        flechas[2].shift(2*alto_rectangulo* UP)
        flechas[3].shift(alto_rectangulo* UP)
        #flechas[4].shift()
        flechas[5].shift(alto_rectangulo* DOWN)
        flechas[6].shift(2*alto_rectangulo* DOWN)
        flechas[7].shift(2.5* alto_rectangulo* DOWN + 0.5*RIGHT).scale(0.76)
        flechas[8].shift(3*alto_rectangulo * DOWN + ancho_rectangulo*RIGHT).scale(0.56)
        flechas[9].shift(3.5*alto_rectangulo * DOWN + 1.5 * ancho_rectangulo*RIGHT).scale(0.35)

        # muevo el grupo de flechas para que no tapen los superíndices
        flechas.shift(0.03 * UP)

        # Crear niveles para el ejemplo: Litio ##################################
        numero_atomico = 3
        titulo_numero_electrones = Text("N° electrones = ", font="Liberation Sans", font_size=32).shift(3.5 * UP+ 0.25*RIGHT)
        titulo_numero_atomico = Text("Número Atómico(Z)", font="Liberation Sans", font_size=32).next_to(titulo_numero_electrones)
        self.play(FadeIn(titulo_numero_electrones), FadeIn(titulo_numero_atomico))
        litio_cuadro = crear_cuadro_elemento("Li", "Litio", numero_atomico, "6.941U", "Metal", "+1", color_fondo="#53AC9E", color_interior="#315941").shift(1.5 * RIGHT)
        self.play(FadeIn(litio_cuadro), run_time=3.5)
        self.wait(9)
        self.play(litio_cuadro[7].animate.scale(2))
        self.play(Circumscribe(litio_cuadro[7]), buffer=0.2)
        self.play(Circumscribe(litio_cuadro[7]), buffer=0.2)
        self.play(litio_cuadro[7].animate.scale(0.5))
        z_litio = Text("3", font="Monospace", font_size=30).next_to(titulo_numero_electrones)
        self.play(Transform(titulo_numero_atomico, z_litio))
        self.play(litio_cuadro.animate.scale(0.4).shift(2.75*UP + 4.55*RIGHT))
        self.wait(14)

        distrib = distribucion_electronica(numero_atomico)
        electrones_por_nivel = contar_e_por_nivel(distrib)
        max_nivel = max(electrones_por_nivel.keys())

        centro_nucleo = nucleo.get_center()
        orbitas_ejemplo = VGroup(*crear_orbitas(max_nivel, centro=centro_nucleo, radio_inicial=0.75, paso=0.45))   #n: int, centro=ORIGIN, radio_inicial: float = 0.55, paso: float = 0.3, color=BLUE
        flechas_copia = VGroup()

        copia_tags = VGroup()
        copia_tag_alineado_1s2=  tags_subniveles_alineados[4].copy().scale(1.2)
        incompleto_dos_s1 = Tex(r"$2s^1$",font_size=38).set(z_index=505).scale(1.2)
        copia_tags.add(copia_tag_alineado_1s2, incompleto_dos_s1)

        # copio flechas
        for i in range(2):
            flechas_copia.add(flechas[i].copy())
        # animo las dos primeras flechas
        self.play(GrowFromCenter(flechas_copia[0]), run_time=1.3)
        self.wait(14)
        copia_tag_alineado_1s2.shift(4*RIGHT + 0.15 * DOWN)
        self.play(FadeIn(copia_tag_alineado_1s2))
        self.wait(5)
        self.play(GrowFromCenter(flechas_copia[1]), run_time=1.3)
        incompleto_dos_s1.next_to(copia_tag_alineado_1s2)
        self.play(FadeIn(incompleto_dos_s1))
        self.wait(38)
        frame_configuracion = SurroundingRectangle(copia_tags, buff=0.1)

        ## Hacer electrones que salgan de debajo de los tags y se ubiquen en los orbitales. Es decir, los muevo hasta donde están los que mostraba originalmente.
        electrones_tags = VGroup()
        for i in range(numero_atomico):
            electron_tag = Circle(radius=0.08, color=color, fill_opacity=0.6, fill_color=color)
            electrones_tags.add(electron_tag)
        electrones_tags[0].shift(copia_tag_alineado_1s2.get_center() + 0.6 * DOWN)
        electrones_tags[1].shift(copia_tag_alineado_1s2.get_center() + 0.9 * DOWN)
        electrones_tags[2].shift(incompleto_dos_s1.get_center() +  0.6 * DOWN)

        # animo los electrones en su posición inicial
        self.play(FadeIn(electrones_tags[0]), FadeIn(electrones_tags[1]))
        self.wait(2)
        self.play(FadeIn(electrones_tags[2]))


        self.play(FadeIn(frame_configuracion), run_time=1.5)

        # vuelvo a mostrar el núcleo
        self.play(nucleo.animate.set_opacity(1), run_time = 2)
        # Creo un grupo para acumular todos los electrones y poder eliminarlos cuando no los necesite
        todos_los_electrones = VGroup()
        etiquetas = VGroup()
        # animo las órbitas
        for i, orbita in enumerate(orbitas_ejemplo):
            # muestro la órbita que corresponde al nivel actual
            self.play(GrowFromCenter(orbita))
            nivel = i + 1
            cantidad = electrones_por_nivel.get(nivel, 0)
            electrones = ubicar_electrones(cantidad, orbita.radius, nucleo.get_center(), GREEN)
            # Agrego los electrones de esa iteración al grupo total
            todos_los_electrones.add(*electrones)
            self.wait(1)


        # Mover los electrones a las órbitas
        for i in range(len(electrones_tags)):
            self.play(electrones_tags[i].animate.move_to(todos_los_electrones[i]), run_time = 0.5)
            nivel = i + 1

        self.wait(22)
         # $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

        # Mostrar etiqueta para cada nivel
        cantidad = electrones_por_nivel.get(nivel, 0)
        offset_y = 0.8
        etiqueta_n1 = crear_etiqueta(2, 1, 3, 0.6).shift(5.5 * RIGHT + offset_y * DOWN)
        etiqueta_n2 = crear_etiqueta(1, 2, 3, 0.6).shift(5.5 * RIGHT + 2 * offset_y * DOWN)
        etiquetas.add(etiqueta_n1, etiqueta_n2)
        self.play(FadeIn(etiqueta_n1), run_time = 1.5)
        self.play(FadeIn(etiqueta_n2), run_time = 1.75)
        self.wait(18)

        # Desaparecemos el modelo de órbitas, los electrones, el cuadro de ejemplo y las flechas
        self.play(ShrinkToCenter(orbitas_ejemplo),
                  nucleo.animate.set_opacity(0),
                  FadeOut(flechas_copia), FadeOut(copia_tag_alineado_1s2), FadeOut(incompleto_dos_s1),
                  FadeOut(etiquetas),FadeOut(frame_configuracion) , ShrinkToCenter(electrones_tags),
                  run_time = 2.5
                  )
        self.wait(2)

#33333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333

class DistribucionElectronicaETres(Scene):

    def construct(self):

        #Añadir logo
        logo = h.Logo()
        self.add(logo)

        ###########################
        #                         #
        #      Tabla niveles      #
        #      y subniveles       #
        #                         #
        ###########################

        ancho_rectangulo = 0.9
        alto_rectangulo = 0.625

      # Crear 8 para la primera columna
        para_niveles = VGroup(*[
            Rectangle(width=ancho_rectangulo, height=alto_rectangulo, color=WHITE)
            for _ in range(8)
        ])

        col_niveles = en_grilla(para_niveles,
            columnas=1,
            espacio_x=ancho_rectangulo,   # Si el epacio que le doy es = al ancho del rectángulo, entonces quedan pegados uno al lado del otro
            espacio_y=alto_rectangulo,
            alineacion_horizontal="izquierda",
            alineacion_vertical="arriba",
            max_izquierda=-6.15,
            max_derecha=6,
            max_abajo=-3.5,
            max_arriba=3.3)


      # Crear 4 columnas para los subniveles
        para_subniveles = VGroup(*[
            Rectangle(width=ancho_rectangulo, height=alto_rectangulo, color=WHITE)
            for _ in range(32)
        ])

        cols_subniveles = en_grilla(para_subniveles,
            columnas=4,
            espacio_x=ancho_rectangulo,
            espacio_y=alto_rectangulo,
            alineacion_horizontal="izquierda",
            alineacion_vertical="arriba",
            max_izquierda=-6.15,
            max_derecha=6,
            max_abajo=-3.5,
            max_arriba=3.3).shift((ancho_rectangulo) * RIGHT)

        tabla = Group(col_niveles, cols_subniveles)
        tabla.shift(0.25*LEFT)

        # Texto a casillas
        tags_niveles= ["","1","2","3","4","5","6","7"]
        tags_niveles_alineados= Group()

        for i in range(len(tags_niveles)):
          tag_nivel = Text(tags_niveles[i], font="Monospace", font_size=20 ).move_to(col_niveles[i].get_center())
          tags_niveles_alineados.add(tag_nivel)
        tabla.add(tags_niveles_alineados)             # Agrego los tags a la tabla


        tags_subniveles = [
                              Tex(r"$s$",font_size=38).set(z_index=600), Tex(r"$p$",font_size=38).set(z_index=601), Tex(r"$d$",font_size=38).set(z_index=602), Tex(r"$f$",font_size=38).set(z_index=603),
                              Tex(r"$1s^2$",font_size=38).set(z_index=605), Tex(r"",font_size=38).set(z_index=606), Tex(r"",font_size=38).set(z_index=607), Tex(r"",font_size=38).set(z_index=608),  # vacíos pero válidos como raw strings
                              Tex(r"$2s^2$",font_size=38).set(z_index=609), Tex(r"$2p^6$",font_size=38).set(z_index=610), Tex(r"",font_size=38).set(z_index=611), Tex(r"",font_size=38).set(z_index=612),
                              Tex(r"$3s^2$",font_size=38).set(z_index=613), Tex(r"$3p^6$",font_size=38).set(z_index=614), Tex(r"$3d^{10}$",font_size=38).set(z_index=615), Tex(r"",font_size=38).set(z_index=616),
                              Tex(r"$4s^2$",font_size=38).set(z_index=617), Tex(r"$4p^6$",font_size=38).set(z_index=618), Tex(r"$4d^{10}$",font_size=38).set(z_index=619), Tex(r"$4f^{14}$",font_size=38).set(z_index=620),
                              Tex(r"$5s^2$",font_size=38).set(z_index=621), Tex(r"$5p^6$",font_size=38).set(z_index=622), Tex(r"$5d^{10}$",font_size=38).set(z_index=623), Tex(r"$5f^{14}$",font_size=38).set(z_index=624),
                              Tex(r"$6s^2$",font_size=38).set(z_index=625), Tex(r"$6p^6$",font_size=38).set(z_index=626), Tex(r"$6d^{10}$",font_size=38).set(z_index=627), Tex(r"",font_size=38).set(z_index=628),
                              Tex(r"$7s^2$",font_size=38).set(z_index=629), Tex(r"$7p^6$",font_size=38).set(z_index=630), Tex(r"",font_size=38).set(z_index=631), Tex(r"",font_size=38).set(z_index=632),
                          ]


        tags_subniveles_alineados = VGroup()
        for i in range(len(tags_subniveles)):
            tag = tags_subniveles[i].copy().move_to(cols_subniveles[i].get_center())
            tags_subniveles_alineados.add(tag)

        tabla.add(tags_subniveles_alineados)

        # Colorear casillas

        colores = ["#CDE0B8", "#E0C1B8", "#B8C1E0", "#5A658B", "#49612F", "#613A2F", "#285FDE", GRAY, "#2A6114", "#B1E09F"]


        for i in range(4, len(cols_subniveles)):  # empezar en 4 para saltar la primera fila
            idx_relativo = i - 4  # posición dentro de la tabla real (sin títulos)
            fila = idx_relativo // 4
            col = idx_relativo % 4
            diagonal = fila + col
            color = colores[diagonal % len(colores)]

            cols_subniveles[i].set_fill(color, opacity=0.5)

        self.add(tabla)

        ###########################
        #                         #
        #      Modelo orbital     #
        #                         #
        ###########################


        nucleo = crear_circulo_sombreado(0.28, RED_B, 0.8, 0.5, WHITE, 1, 0).shift(1.15 * DOWN + 1.15*RIGHT)
        ##self.play(GrowFromCenter(nucleo))

        contador = 1
        orbitas_muestras = VGroup(*crear_orbitas(7, nucleo.get_center(), 0.6, 0.3, color=BLUE))
        niveles_permanentes= Group()

        flechas = VGroup()
        flecha_original = Arrow(start=cols_subniveles[11].get_corner(UR),end=col_niveles[6].get_corner(DL),stroke_width=3,
                                   stroke_opacity=1, fill_opacity=1)  # aplica a la punta)
        #self.add(flecha_original)

        for i in range(len(colores)):
            flecha_temp = flecha_original.copy()
            flechas.add(flecha_temp)
        # Agrego las flechas al grupo
        flechas[0].shift(3*alto_rectangulo * UP + ancho_rectangulo*LEFT).scale(0.575)
        flechas[1].shift(2.5* alto_rectangulo* UP + 0.5 * ancho_rectangulo * LEFT ).scale(0.8)
        flechas[2].shift(2*alto_rectangulo* UP)
        flechas[3].shift(alto_rectangulo* UP)
        #flechas[4].shift()
        flechas[5].shift(alto_rectangulo* DOWN)
        flechas[6].shift(2*alto_rectangulo* DOWN)
        flechas[7].shift(2.5* alto_rectangulo* DOWN + 0.5*RIGHT).scale(0.76)
        flechas[8].shift(3*alto_rectangulo * DOWN + ancho_rectangulo*RIGHT).scale(0.56)
        flechas[9].shift(3.5*alto_rectangulo * DOWN + 1.5 * ancho_rectangulo*RIGHT).scale(0.35)

        # muevo el grupo de flechas para que no tapen los superíndices
        flechas.shift(0.03 * UP)

        # Crear niveles para el ejemplo: Litio ##################################

        numero_atomico = 3
        titulo_numero_electrones = Text("N° electrones = ", font="Liberation Sans", font_size=32).shift(3.5 * UP+ 0.25*RIGHT)
        titulo_numero_atomico = Text("Número Atómico(Z)", font="Liberation Sans", font_size=32).next_to(titulo_numero_electrones)

        litio_cuadro = crear_cuadro_elemento("Li", "Litio", numero_atomico, "6.941U", "Metal", "+1", color_fondo="#53AC9E", color_interior="#315941").shift(1.5 * RIGHT)

        z_litio = Text("3", font="Monospace", font_size=30).next_to(titulo_numero_electrones)
        litio_cuadro.scale(0.4).shift(2.75*UP + 4.55*RIGHT)
        self.add(titulo_numero_electrones, z_litio, litio_cuadro)

        copia_tags = VGroup()

        copia_tag_alineado_1s2=  tags_subniveles_alineados[4].copy().scale(1.2)
        copia_tag_alineado_2s2 = tags_subniveles_alineados[8].copy().scale(1.2)
        incompleto_dos_p4 = Tex(r"$2p^4$",font_size=38).set(z_index=506).scale(1.2)
        copia_tags.add(copia_tag_alineado_1s2, copia_tag_alineado_2s2, incompleto_dos_p4)

        # Crear niveles para el ejemplo Oxígeno #################################
        # Corremos el centro del núcleo para no recargar la derecha del frame
        nucleo.shift(0.6*LEFT)
        centro_nucleo = nucleo.get_center()

        numero_atomico = 8
        oxigeno_cuadro = crear_cuadro_elemento("O", "Oxígeno", numero_atomico, "15.999U", "No Metal", "-1 -2").shift(1.5 * RIGHT)
        self.play(FadeIn(oxigeno_cuadro), run_time=3)
        self.wait(1)
        self.play(oxigeno_cuadro[7].animate.scale(2))
        self.play(Circumscribe(oxigeno_cuadro[7]), buffer=0.2)
        self.play(Circumscribe(oxigeno_cuadro[7]), buffer=0.2)
        self.play(oxigeno_cuadro[7].animate.scale(0.5))

        z_oxigeno = Text("8", font="Monospace", font_size=30).next_to(titulo_numero_electrones)
        self.play(Transform(z_litio, z_oxigeno))
        self.play(oxigeno_cuadro.animate.scale(0.4).shift(2.75*UP + 4.40*RIGHT))

        distrib = distribucion_electronica(numero_atomico)
        electrones_por_nivel = contar_e_por_nivel(distrib)
        max_nivel = max(electrones_por_nivel.keys())

        centro_nucleo = nucleo.get_center()
        orbitas_ejemplo = VGroup(*crear_orbitas(max_nivel, centro=centro_nucleo, radio_inicial=0.75, paso=0.45))

        flechas_copia = VGroup()
        # copio flechas y agrego al grupo
        for i in range(3):
            flechas_copia.add(flechas[i].copy())
        # animo las tres primeras flechas
        self.play(GrowFromCenter(flechas_copia[0]), run_time=1.25)
        self.wait(1)
        copia_tag_alineado_1s2.shift(4*RIGHT + 0.15 * DOWN)
        self.play(FadeIn(copia_tag_alineado_1s2))
        self.wait(3)
        self.play(GrowFromCenter(flechas_copia[1]), run_time=1.3)
        copia_tag_alineado_2s2.next_to(copia_tag_alineado_1s2)
        self.wait(3)
        self.play(FadeIn(copia_tag_alineado_2s2))
        self.wait(9)
        self.play(GrowFromCenter(flechas_copia[2]), run_time=1.3)
        self.wait(16)
        incompleto_dos_p4.next_to(copia_tag_alineado_2s2)
        self.play(FadeIn(incompleto_dos_p4))
        self.wait(8)
        frame_configuracion = SurroundingRectangle(copia_tags, buff=0.1)


        ## Hacer electrones que salgan de debajo de los tags y se ubiquen en los orbitales. Es decir, los muevo hasta donde están los que mostraba originalmente.
        electrones_tags = VGroup()
        for i in range(numero_atomico):
            electron_tag = Circle(radius=0.08, color=color, fill_opacity=0.6, fill_color=color)
            electrones_tags.add(electron_tag)
        electrones_tags[0].shift(copia_tag_alineado_1s2.get_center() + 0.6 * DOWN)
        electrones_tags[1].shift(copia_tag_alineado_1s2.get_center() + 0.9 * DOWN)
        electrones_tags[2].shift(copia_tag_alineado_2s2.get_center() + 0.6 * DOWN)
        electrones_tags[3].shift(copia_tag_alineado_2s2.get_center() + 0.9 * DOWN)
        electrones_tags[4].shift(incompleto_dos_p4.get_center() +  0.6 * DOWN + 0.14 * LEFT)
        electrones_tags[5].shift(incompleto_dos_p4.get_center() +  0.9 * DOWN + 0.14 * LEFT)
        electrones_tags[6].shift(incompleto_dos_p4.get_center() +  0.6 * DOWN + 0.14 * RIGHT)
        electrones_tags[7].shift(incompleto_dos_p4.get_center() +  0.9 * DOWN + 0.14 * RIGHT)

        # animo los electrones en su posición inicial
        self.play(FadeIn(electrones_tags[0]), FadeIn(electrones_tags[1]))
        self.wait(2)
        self.play(FadeIn(electrones_tags[2]), FadeIn(electrones_tags[3]))
        self.wait(2)
        self.play(FadeIn(electrones_tags[4]), FadeIn(electrones_tags[5]))
        self.wait(1)
        self.play(FadeIn(electrones_tags[6]), FadeIn(electrones_tags[7]))

        self.play(FadeIn(frame_configuracion), run_time=1.5)
         # vuelvo a mostrar el núcleo
        self.play(nucleo.animate.set_opacity(1), run_time = 1.5)
        # Creo un grupo para acumular todos los electrones y poder eliminarlos cuando no los necesite
        todos_los_electrones = VGroup()
        etiquetas = VGroup()
        # animo las órbitas
        for i, orbita in enumerate(orbitas_ejemplo):
            # muestro la órbita que corresponde al nivel actual
            self.play(GrowFromCenter(orbita))
            nivel = i + 1
            cantidad = electrones_por_nivel.get(nivel, 0)
            electrones = ubicar_electrones(cantidad, orbita.radius, nucleo.get_center(), GREEN)
            # Agrego los electrones de esa iteración al grupo total
            todos_los_electrones.add(*electrones)
            self.wait(1)




        # Mover los electrones a las órbitas
        for i in range(len(electrones_tags)):
            self.play(electrones_tags[i].animate.move_to(todos_los_electrones[i]), run_time=0.55)
            nivel = i + 1

        self.wait(3)

        # Mostrar etiqueta para cada nivel
        cantidad = electrones_por_nivel.get(nivel, 0)
        offset_y = 0.8
        etiqueta_n1 = crear_etiqueta(2, 1, 3, 0.6).shift(5.5 * RIGHT + offset_y * DOWN)
        etiqueta_n2 = crear_etiqueta(6, 2, 3, 0.6).shift(5.5 * RIGHT + 2 * offset_y * DOWN)
        etiquetas.add(etiqueta_n1, etiqueta_n2)
        self.play(FadeIn(etiqueta_n1), run_time = 1.5)
        self.play(FadeIn(etiqueta_n2), run_time = 1.75)
        self.wait(4)

       # Desaparecemos el modelo de órbitas
        self.play(ShrinkToCenter(orbitas_ejemplo), #ShrinkToCenter(todos_los_electrones),
                  nucleo.animate.set_opacity(0), FadeOut(flechas_copia),FadeOut(copia_tag_alineado_1s2),
                  FadeOut(copia_tag_alineado_2s2), FadeOut(incompleto_dos_p4),
                  FadeOut(etiquetas), FadeOut(frame_configuracion),
                  ShrinkToCenter(electrones_tags), run_time = 2.75)


        self.wait(3)

#4444444444444444444444444444444444444444444444444444444444444444444444444444444444444444444

class DistribucionElectronicaECuatro(Scene):

    def construct(self):

        #Añadir logo
        logo = h.Logo()
        self.add(logo)

        ###########################
        #                         #
        #      Tabla niveles      #
        #      y subniveles       #
        #                         #
        ###########################

        ancho_rectangulo = 0.9
        alto_rectangulo = 0.625

      # Crear 8 para la primera columna
        para_niveles = VGroup(*[
            Rectangle(width=ancho_rectangulo, height=alto_rectangulo, color=WHITE)
            for _ in range(8)
        ])

        col_niveles = en_grilla(para_niveles,
            columnas=1,
            espacio_x=ancho_rectangulo,   # Si el epacio que le doy es = al ancho del rectángulo, entonces quedan pegados uno al lado del otro
            espacio_y=alto_rectangulo,
            alineacion_horizontal="izquierda",
            alineacion_vertical="arriba",
            max_izquierda=-6.15,
            max_derecha=6,
            max_abajo=-3.5,
            max_arriba=3.3)


      # Crear 4 columnas para los subniveles
        para_subniveles = VGroup(*[
            Rectangle(width=ancho_rectangulo, height=alto_rectangulo, color=WHITE)
            for _ in range(32)
        ])

        cols_subniveles = en_grilla(para_subniveles,
            columnas=4,
            espacio_x=ancho_rectangulo,
            espacio_y=alto_rectangulo,
            alineacion_horizontal="izquierda",
            alineacion_vertical="arriba",
            max_izquierda=-6.15,
            max_derecha=6,
            max_abajo=-3.5,
            max_arriba=3.3).shift((ancho_rectangulo) * RIGHT)

        tabla = Group(col_niveles, cols_subniveles)
        tabla.shift(0.25*LEFT)

        # Texto a casillas
        tags_niveles= ["","1","2","3","4","5","6","7"]
        tags_niveles_alineados= Group()

        for i in range(len(tags_niveles)):
          tag_nivel = Text(tags_niveles[i], font="Monospace", font_size=20 ).move_to(col_niveles[i].get_center())
          tags_niveles_alineados.add(tag_nivel)
        tabla.add(tags_niveles_alineados)             # Agrego los tags a la tabla


        tags_subniveles = [
                              Tex(r"$s$",font_size=38).set(z_index=600), Tex(r"$p$",font_size=38).set(z_index=601), Tex(r"$d$",font_size=38).set(z_index=602), Tex(r"$f$",font_size=38).set(z_index=603),
                              Tex(r"$1s^2$",font_size=38).set(z_index=605), Tex(r"",font_size=38).set(z_index=606), Tex(r"",font_size=38).set(z_index=607), Tex(r"",font_size=38).set(z_index=608),  # vacíos pero válidos como raw strings
                              Tex(r"$2s^2$",font_size=38).set(z_index=609), Tex(r"$2p^6$",font_size=38).set(z_index=610), Tex(r"",font_size=38).set(z_index=611), Tex(r"",font_size=38).set(z_index=612),
                              Tex(r"$3s^2$",font_size=38).set(z_index=613), Tex(r"$3p^6$",font_size=38).set(z_index=614), Tex(r"$3d^{10}$",font_size=38).set(z_index=615), Tex(r"",font_size=38).set(z_index=616),
                              Tex(r"$4s^2$",font_size=38).set(z_index=617), Tex(r"$4p^6$",font_size=38).set(z_index=618), Tex(r"$4d^{10}$",font_size=38).set(z_index=619), Tex(r"$4f^{14}$",font_size=38).set(z_index=620),
                              Tex(r"$5s^2$",font_size=38).set(z_index=621), Tex(r"$5p^6$",font_size=38).set(z_index=622), Tex(r"$5d^{10}$",font_size=38).set(z_index=623), Tex(r"$5f^{14}$",font_size=38).set(z_index=624),
                              Tex(r"$6s^2$",font_size=38).set(z_index=625), Tex(r"$6p^6$",font_size=38).set(z_index=626), Tex(r"$6d^{10}$",font_size=38).set(z_index=627), Tex(r"",font_size=38).set(z_index=628),
                              Tex(r"$7s^2$",font_size=38).set(z_index=629), Tex(r"$7p^6$",font_size=38).set(z_index=630), Tex(r"",font_size=38).set(z_index=631), Tex(r"",font_size=38).set(z_index=632),
                          ]

        tags_subniveles_alineados = VGroup()
        for i in range(len(tags_subniveles)):
            tag = tags_subniveles[i].copy().move_to(cols_subniveles[i].get_center())
            tags_subniveles_alineados.add(tag)

        tabla.add(tags_subniveles_alineados)

        # Colorear casillas

        colores = ["#CDE0B8", "#E0C1B8", "#B8C1E0", "#5A658B", "#49612F", "#613A2F", "#285FDE", GRAY, "#2A6114", "#B1E09F"]


        for i in range(4, len(cols_subniveles)):  # empezar en 4 para saltar la primera fila
            idx_relativo = i - 4  # posición dentro de la tabla real (sin títulos)
            fila = idx_relativo // 4
            col = idx_relativo % 4
            diagonal = fila + col
            color = colores[diagonal % len(colores)]

            cols_subniveles[i].set_fill(color, opacity=0.5)

        self.add(tabla)

        ###########################
        #                         #
        #      Modelo orbital     #
        #                         #
        ###########################

        nucleo = crear_circulo_sombreado(0.28, RED_B, 0.8, 0.5, WHITE, 1, 0).shift(1.15 * DOWN + 1.15*RIGHT)
        ##self.play(GrowFromCenter(nucleo))

        contador = 1
        orbitas_muestras = VGroup(*crear_orbitas(7, nucleo.get_center(), 0.6, 0.3, color=BLUE))
        niveles_permanentes= Group()

        flechas = VGroup()
        flecha_original = Arrow(start=cols_subniveles[11].get_corner(UR),end=col_niveles[6].get_corner(DL),stroke_width=3,
                                   stroke_opacity=1, fill_opacity=1)  # aplica a la punta)
        #self.add(flecha_original)

        for i in range(len(colores)):
            flecha_temp = flecha_original.copy()
            flechas.add(flecha_temp)
        # Agrego las flechas al grupo
        flechas[0].shift(3*alto_rectangulo * UP + ancho_rectangulo*LEFT).scale(0.575)
        flechas[1].shift(2.5* alto_rectangulo* UP + 0.5 * ancho_rectangulo * LEFT ).scale(0.8)
        flechas[2].shift(2*alto_rectangulo* UP)
        flechas[3].shift(alto_rectangulo* UP)
        #flechas[4].shift()
        flechas[5].shift(alto_rectangulo* DOWN)
        flechas[6].shift(2*alto_rectangulo* DOWN)
        flechas[7].shift(2.5* alto_rectangulo* DOWN + 0.5*RIGHT).scale(0.76)
        flechas[8].shift(3*alto_rectangulo * DOWN + ancho_rectangulo*RIGHT).scale(0.56)
        flechas[9].shift(3.5*alto_rectangulo * DOWN + 1.5 * ancho_rectangulo*RIGHT).scale(0.35)

        # muevo el grupo de flechas para que no tapen los superíndices
        flechas.shift(0.03 * UP)

        # Crear niveles para el ejemplo: Litio ##################################

        numero_atomico = 3
        titulo_numero_electrones = Text("N° electrones = ", font="Liberation Sans", font_size=32).shift(3.5 * UP+ 0.25*RIGHT)
        titulo_numero_atomico = Text("Número Atómico(Z)", font="Liberation Sans", font_size=32).next_to(titulo_numero_electrones)

        litio_cuadro = crear_cuadro_elemento("Li", "Litio", numero_atomico, "6.941U", "Metal", "+1", color_fondo="#53AC9E", color_interior="#315941").shift(1.5 * RIGHT)

        z_litio = Text("3", font="Monospace", font_size=30).next_to(titulo_numero_electrones)
        litio_cuadro.scale(0.4).shift(2.75*UP + 4.55*RIGHT)
        self.add(titulo_numero_electrones, litio_cuadro)


        centro_nucleo = nucleo.get_center()
        copia_tags = VGroup()

        copia_tag_alineado_1s2=  tags_subniveles_alineados[4].copy().scale(1.2)
        copia_tag_alineado_2s2 = tags_subniveles_alineados[8].copy().scale(1.2)
        incompleto_dos_s1 = Tex(r"$2s^1$",font_size=38).set(z_index=505).scale(1.2)
        copia_tag_alineado_2p6 = tags_subniveles_alineados[9].copy().scale(1.2)
        incompleto_dos_p4 = Tex(r"$2p^4$",font_size=38).set(z_index=506).scale(1.2)
        copia_tag_alineado_3s2 = tags_subniveles_alineados[12].copy().scale(1.2)
        copia_tag_alineado_3p6 = tags_subniveles_alineados[13].copy().scale(1.2)
        copia_tag_alineado_3d10 = tags_subniveles_alineados[14].copy().scale(1.2)
        incompleto_tres_d10 = Tex(r"$3d^3$",font_size=38).set(z_index=507).scale(1.2)
        copia_tag_alineado_4s2  = tags_subniveles_alineados[16].copy().scale(1.2)
        copia_tags.add(copia_tag_alineado_1s2, copia_tag_alineado_2s2, copia_tag_alineado_2p6, copia_tag_alineado_3s2, copia_tag_alineado_3p6,
                       incompleto_tres_d10, copia_tag_alineado_4s2)

        # Crear niveles para el ejemplo Oxígeno #################################
        numero_atomico = 8
        oxigeno_cuadro = crear_cuadro_elemento("O", "Oxígeno", numero_atomico, "15.999U", "No Metal", "-1 -2").shift(1.5 * RIGHT)


        z_oxigeno = Text("8", font="Monospace", font_size=30).next_to(titulo_numero_electrones)
        self.add(z_oxigeno)
        oxigeno_cuadro.scale(0.4).shift(2.75*UP + 4.40*RIGHT)
        self.add(oxigeno_cuadro)


        # Crear niveles para el ejemplo Vanadio
        # Corremos el centro del núcleo para no recargar la derecha del frame
        nucleo.shift(0.6*LEFT)
        centro_nucleo = nucleo.get_center()

        numero_atomico = 23
        vanadio_cuadro = crear_cuadro_elemento("V", "Vanadio", numero_atomico, "50.942U", "Metal de t.", "+2+3+4+5", color_fondo="#BD428C", color_interior="#713053").shift(1.5 * RIGHT)
        vanadio_cuadro[4].shift(0.1*RIGHT)  # Rectifico la string del tipo porque está quedando afuera y no quiero tocar la función.
        self.play(FadeIn(vanadio_cuadro), run_time=3)
        self.wait(1)
        self.play(vanadio_cuadro[7].animate.scale(2))
        self.play(Circumscribe(vanadio_cuadro[7]), buff=0.2)
        self.play(Circumscribe(vanadio_cuadro[7]), buff=0.2)
        self.play(vanadio_cuadro[7].animate.scale(0.5))
        z_vanadio = Text("23", font="Monospace", font_size=30).next_to(titulo_numero_electrones)
        self.play(Transform(z_oxigeno, z_vanadio))
        self.play(vanadio_cuadro.animate.scale(0.4).shift(2.75*UP + 4.25*RIGHT))
        self.wait(15)

        distrib = distribucion_electronica(numero_atomico)
        electrones_por_nivel = contar_e_por_nivel(distrib)
        max_nivel = max(electrones_por_nivel.keys())


        orbitas_ejemplo = VGroup(*crear_orbitas(max_nivel, centro=centro_nucleo, radio_inicial=0.75, paso=0.45))

        flechas_copia = VGroup()
        # copio flechas
        for i in range(5):
            flechas_copia.add(flechas[i].copy())

        # animo las dos primeras flechas
        self.play(GrowFromCenter(flechas_copia[0]), run_time = 1)
        self.wait(1)
        copia_tag_alineado_1s2.shift(4*RIGHT + 0.15 * DOWN)
        self.play(FadeIn(copia_tag_alineado_1s2))
        self.wait(2)
        self.play(GrowFromCenter(flechas_copia[1]), run_time=1)
        copia_tag_alineado_2s2.next_to(copia_tag_alineado_1s2)
        self.play(FadeIn(copia_tag_alineado_2s2))
        self.wait(16)
        self.play(GrowFromCenter(flechas_copia[2]), run_time=1.25)
        copia_tag_alineado_2p6.next_to(copia_tag_alineado_2s2)
        self.play(FadeIn(copia_tag_alineado_2p6))
        self.wait(7)
        copia_tag_alineado_3s2.next_to(copia_tag_alineado_2p6)
        self.play(FadeIn(copia_tag_alineado_3s2))
        self.wait(2)
        self.play(GrowFromCenter(flechas_copia[3]), run_time = 1.2)
        copia_tag_alineado_3p6.next_to(copia_tag_alineado_3s2)
        self.play(FadeIn(copia_tag_alineado_3p6))
        self.wait(3)
        copia_tag_alineado_4s2.next_to(copia_tag_alineado_3p6)
        self.play(FadeIn(copia_tag_alineado_4s2))
        self.wait(3)
        self.play(GrowFromCenter(flechas_copia[4]), run_time=1.2)
        incompleto_tres_d10.next_to(copia_tag_alineado_4s2)
        self.play(FadeIn(incompleto_tres_d10))
        self.wait(6)
        frame_configuracion = SurroundingRectangle(copia_tags, buff=0.1)


        ## Hacer electrones que salgan de debajo de los tags y se ubiquen en los orbitales. Es decir, los muevo hasta donde están los que mostraba originalmente.
        electrones_tags = VGroup()
        for i in range(numero_atomico):
            electron_tag = Circle(radius=0.08, color=color, fill_opacity=0.6, fill_color=color)
            electrones_tags.add(electron_tag)
        electrones_tags[0].shift(copia_tag_alineado_1s2.get_center() + 0.6 * DOWN)
        electrones_tags[1].shift(copia_tag_alineado_1s2.get_center() + 0.9 * DOWN)
        electrones_tags[2].shift(copia_tag_alineado_2s2.get_center() + 0.6 * DOWN)
        electrones_tags[3].shift(copia_tag_alineado_2s2.get_center() + 0.9 * DOWN)
        electrones_tags[4].shift(copia_tag_alineado_2p6.get_center() +  0.6 * DOWN + 0.14 * LEFT)
        electrones_tags[5].shift(copia_tag_alineado_2p6.get_center() +  0.9 * DOWN + 0.14 * LEFT)
        electrones_tags[6].shift(copia_tag_alineado_2p6.get_center() +  1.2 * DOWN + 0.14 * LEFT)
        electrones_tags[7].shift(copia_tag_alineado_2p6.get_center() +  0.6 * DOWN + 0.14 * RIGHT)
        electrones_tags[8].shift(copia_tag_alineado_2p6.get_center() +  0.9 * DOWN + 0.14 * RIGHT)
        electrones_tags[9].shift(copia_tag_alineado_2p6.get_center() +  1.2 * DOWN + 0.14 * RIGHT)
        electrones_tags[10].shift(copia_tag_alineado_3s2.get_center() + 0.6 * DOWN)
        electrones_tags[11].shift(copia_tag_alineado_3s2.get_center() + 0.9 * DOWN)
        electrones_tags[12].shift(copia_tag_alineado_3p6.get_center() +  0.6 * DOWN + 0.14 * LEFT)
        electrones_tags[13].shift(copia_tag_alineado_3p6.get_center() +  0.9 * DOWN + 0.14 * LEFT)
        electrones_tags[14].shift(copia_tag_alineado_3p6.get_center() +  1.2 * DOWN + 0.14 * LEFT)
        electrones_tags[15].shift(copia_tag_alineado_3p6.get_center() +  0.6 * DOWN + 0.14 * RIGHT)
        electrones_tags[16].shift(copia_tag_alineado_3p6.get_center() +  0.9 * DOWN + 0.14 * RIGHT)
        electrones_tags[17].shift(copia_tag_alineado_3p6.get_center() +  1.2 * DOWN + 0.14 * RIGHT)
        electrones_tags[18].shift(copia_tag_alineado_4s2.get_center() + 0.6 * DOWN)
        electrones_tags[19].shift(copia_tag_alineado_4s2.get_center() + 0.9 * DOWN)
        electrones_tags[20].shift(incompleto_tres_d10.get_center() +  0.6 * DOWN + 0.14 * LEFT)
        electrones_tags[21].shift(incompleto_tres_d10.get_center() +  0.9 * DOWN + 0.14 * LEFT)
        electrones_tags[22].shift(incompleto_tres_d10.get_center() +  0.6 * DOWN + 0.14 * RIGHT)

        primeros_ocho_elec = VGroup(electrones_tags[0], electrones_tags[1], electrones_tags[2],
                                    electrones_tags[3], electrones_tags[4], electrones_tags[5], electrones_tags[6], electrones_tags[7])
        nivel_3p6_elecs = VGroup(electrones_tags[12], electrones_tags[13], electrones_tags[14], electrones_tags[15], electrones_tags[16], electrones_tags[17])

        # animo los electrones en su posición inicial
        self.play(FadeIn(electrones_tags[0]), FadeIn(electrones_tags[1]), run_time=0.75)

        self.play(FadeIn(electrones_tags[2]), FadeIn(electrones_tags[3]), run_time=0.75)

        self.play(FadeIn(electrones_tags[4]), FadeIn(electrones_tags[5]),
                  FadeIn(electrones_tags[6]), FadeIn(electrones_tags[7]),
                  FadeIn(electrones_tags[8]), FadeIn(electrones_tags[9], run_time=0.75))


        self.play(FadeIn(electrones_tags[10]), FadeIn(electrones_tags[11]), run_time=0.75)

        self.play(FadeIn(nivel_3p6_elecs))

        self.play(FadeIn(electrones_tags[18]), FadeIn(electrones_tags[19]), run_time=0.75)

        self.play(FadeIn(electrones_tags[20]), FadeIn(electrones_tags[21]), FadeIn(electrones_tags[22]), run_time=0.75)


        self.play(FadeIn(frame_configuracion), run_time=1)

         # vuelvo a mostrar el núcleo
        self.play(nucleo.animate.set_opacity(1), run_time = 1.25)
        # Creo un grupo para acumular todos los electrones y poder eliminarlos cuando no los necesite
        todos_los_electrones = VGroup()
        etiquetas = VGroup()
        # animo las órbitas
        for i, orbita in enumerate(orbitas_ejemplo):
            # muestro la órbita que corresponde al nivel actual
            self.play(GrowFromCenter(orbita))
            nivel = i + 1
            cantidad = electrones_por_nivel.get(nivel, 0)
            electrones = ubicar_electrones(cantidad, orbita.radius, nucleo.get_center(), GREEN)
            # Agrego los electrones de esa iteración al grupo total
            todos_los_electrones.add(*electrones)
            self.wait(0.5)

        # Mover los electrones a las órbitas
        for i in range(len(electrones_tags)-5):
            self.play(electrones_tags[i].animate.move_to(todos_los_electrones[i]), run_time = 0.75)

            nivel = i + 1
        # acomodar los últimos 5 electrones
        self.play(electrones_tags[18].animate.move_to(todos_los_electrones[-2]), run_time=0.5)
        self.play(electrones_tags[19].animate.move_to(todos_los_electrones[-1]), run_time=0.5)
        self.play(electrones_tags[20].animate.move_to(todos_los_electrones[-3]), run_time=0.5)
        self.play(electrones_tags[21].animate.move_to(todos_los_electrones[-4]), run_time=0.5)
        self.play(electrones_tags[22].animate.move_to(todos_los_electrones[-5]), run_time=0.5)

        self.wait(1)

        # Mostrar etiqueta para cada nivel
        cantidad = electrones_por_nivel.get(nivel, 0)
        offset_y = 0.8
        etiqueta_n1 = crear_etiqueta(2, 1, 3, 0.6).shift(5.5 * RIGHT + offset_y * DOWN)
        etiqueta_n2 = crear_etiqueta(8, 2, 3, 0.6).shift(5.5 * RIGHT + 2 * offset_y * DOWN)
        etiqueta_n3 = crear_etiqueta(11, 3, 3, 0.6).shift(5.5 * RIGHT + 3 * offset_y * DOWN)
        etiqueta_n4 = crear_etiqueta(2, 4, 3, 0.6).shift(5.5 * RIGHT + 4 * offset_y * DOWN)
        etiquetas.add(etiqueta_n1, etiqueta_n2, etiqueta_n3, etiqueta_n4)
        self.play(FadeIn(etiqueta_n1), run_time = 1.2)
        self.play(FadeIn(etiqueta_n2), run_time = 1.25)
        self.play(FadeIn(etiqueta_n3), run_time = 1.25)
        self.play(FadeIn(etiqueta_n4), run_time = 1.5)

        self.wait(28)

        # Desaparecemos el modelo de órbitas
        self.play(ShrinkToCenter(orbitas_ejemplo), ShrinkToCenter(electrones_tags),
                  nucleo.animate.set_opacity(0), FadeOut(flechas_copia),
                  FadeOut(copia_tag_alineado_1s2),FadeOut(copia_tag_alineado_2s2),
                  FadeOut(copia_tag_alineado_2p6),FadeOut(copia_tag_alineado_3s2),
                  FadeOut(copia_tag_alineado_3p6),FadeOut(copia_tag_alineado_4s2),
                  FadeOut(incompleto_tres_d10),
                  FadeOut(etiquetas), FadeOut(tabla), FadeOut(frame_configuracion),
                  FadeOut(litio_cuadro),FadeOut(vanadio_cuadro),FadeOut(oxigeno_cuadro),
                  FadeOut(titulo_numero_electrones), FadeOut(z_oxigeno), FadeOut(copia_tags), run_time=3)

        self.wait(3)

# Retoques

